<a href="https://colab.research.google.com/github/MoacirChrist/phpexcel/blob/master/09_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Geração de Texto
Redes neurais profundas (RNP) são uma técnica de aprendizado profundo que utiliza redes neurais com várias camadas ocultas para processar dados complexos. Essas redes são capazes de aprender padrões e relações semânticas em grandes conjuntos de dados, o que as torna muito úteis para tarefas de processamento de linguagem natural (PLN).

A tarefa de geração de texto utilizando redes neurais profundas (RNP) é um processo em que as redes neurais são treinadas para gerar texto coerente e relevante em resposta a entradas específicas. Isso é alcançado por meio do pré-treinamento com grandes volumes de texto, que permite às redes aprender padrões e relações semânticas entre as palavras.

Uma das formas mais simples de se realizar essa tarefa é treinando uma Rede Neural para, com base em um conjunto de palavras, prever qual será a próxima palavra no texto. Por exemplo, nós poderíamos treinar uma rede para dada a entrada "O cachorro gosta de", a rede nos retornasse a palavra "latir" ou "pular", por exemplo.

Na aula da semana, nós vamos implementar uma Rede Neural Profunda, utilizando uma arquitetura similar à utilizada na aula passada. No entanto, vamos adaptar nossa rede para gerar textos.

Essa adaptação envolve, principalmente, substituir a última camada da nossa rede por uma camada com ativação pela função Softmax. Essa camada vai possuir um neurônio para cada palavra que desejamos que possa ser gerada pelo nosso sistema. Sua saída vai representar a probabilidade de que aquela palavra possa ser incluída no texto gerado.

A funcionalidade que vamos implementar vai ser similar à funcionalidade de autocomplete, no entanto com textos que vamos dar como entrada para o nosso modelo.

##Acquisição do córpus

O córpus que vamos utilizar nesse caderno é um conjunto de regulamentos da nossa universidade disponíveis na seguinte URL:
[Regulamentos da UTFPR](https://www.utfpr.edu.br/documentos/graduacao-e-educacao-profissional/prograd/diretrizes-e-regulamentos).

Inicialmente, nós vamos implementar um script para fazer o download desses regulamentos disponibilizados em um servidor separado.

In [ ]:
import io, tarfile, requests, os, pandas as pd


# download the dataset
def download (url, filename=''):
  if (os.path.isfile(filename)):
    print('Arquivo já existente no Runtime... Tudo OK')
    return
  response = requests.get(url)
  with open(f'./{filename}', 'wb') as f:
      f.write(response.content)
      print('Download realizado e arquivo extraído no Runtime... Tudo OK')

urls = [
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/aa-ab-cf-dispensa-2021.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/ac-2022.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/diretrizes-grad-2022.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/ead-2022.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/estagio-2020.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/extensao-2022.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/rodp-2019.html",
  "https://raw.githubusercontent.com/watinha/nlp-text-mining-datasets/main/regulamentos/tcc-2022.html"
]

filenames = []

for url in urls:
  filename = url.split('/')[-1]
  filenames.append(filename)
  download(url, filename)



Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK
Download realizado e arquivo extraído no Runtime... Tudo OK


Esses arquivos são disponibilizados como páginas HTML. No entanto, para simplificar nossa abordagem de geração de textos, nós podemos remover a estrutura HTML e conteúdos que não são relevantes para gerar os textos.

Nesse contexto, nós podemos utilizar a biblioteca [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/) e localizar apenas os conteúdos textuais relevantes dentro do código das páginas HTML.

Nesses regulamentos, de forma simplificada, podemos considerar que os textos mais relevantes estão localizados dentro de elementos HTML P (parágrafo) com a classe "Texto_Justificado". Por isso, utilizamos a biblioteca Beautiful Soup para localizar esses elementos e extrair os conteúdos textuais de dentro deles para gerarmos o nosso córpus de estudo.

In [ ]:
import codecs

from bs4 import BeautifulSoup


corpus = []

for filename in filenames:
  with codecs.open(filename, encoding='cp1252') as f:
    html = f.read()
    soup = BeautifulSoup(html)
    ps = soup.select('div[unselectable=on] ~ p')
    article = ''

    for p in ps:
      if p.get_text().lower().startswith('art.'):
        article = p.get_text()
        corpus.append(article)
      else:
        paragraph = p.get_text()
        corpus.append(f'{article} {paragraph}')


print('\n - '.join([ doc[:50] for doc in corpus[:10]]))

 ANEXO DA RESOLUÇÃO COGEP/UTFPR Nº 110, DE 19 DE O
 -   
 -  Regulamento para 
as atividades acompanhadas, o a
 -   
 -   
 -  Capítulo I
 -  Das Atividades Acompanhadas
 - Art. 1º  As atividades acompanhadas 
caracterizam-
 - Art. 2º  Poderão solicitar a 
realização de ativid
 - Art. 2º  Poderão solicitar a 
realização de ativid


##Construção do Dataset
Para construir nosso dataset e simplificar nossos experimentos, vamos limpar caractéres desconhecidos e pontuações. No código a seguir, executamos a remoção de diferentes tipos de pontuações e formas de definir espaços em textos.

In [ ]:
import re


def clean(doc):
  words = doc.split()
  chars_to_replace = '!"#$%&\'()*+,-:;<=>?@[\\]^_`{|}~'
  table = doc.maketrans(chars_to_replace, ' ' * len(chars_to_replace))
  cleaned_words = [w.translate(table) for w in words]
  cleaned_doc = ' '.join(cleaned_words)
  cleaned_doc = cleaned_doc.replace(u'\xa0', u' ')
  cleaned_doc = cleaned_doc.replace(u'\u200b', u' ')
  cleaned_doc = cleaned_doc.replace(u'\n', u' ')
  cleaned_doc = re.sub(r'\s+', ' ', cleaned_doc)
  cleaned_doc = cleaned_doc.lower().lstrip()

  return cleaned_doc


corpus = [ clean(doc) for doc in corpus ]
print('\n - '.join([ doc[:50] for doc in corpus[:10]]))



anexo da resolução cogep/utfpr nº 110 de 19 de out
 - 
 - regulamento para as atividades acompanhadas o abon
 - 
 - 
 - capítulo i
 - das atividades acompanhadas
 - art. 1º as atividades acompanhadas caracterizam se
 - art. 2º poderão solicitar a realização de atividad
 - art. 2º poderão solicitar a realização de atividad


No momento, nós temos um conjunto de strings armazenadas na nossa variável corpus. Para definir o nosso dataset, vamos precisar separar esse texto em entrada e classe (label). Para isso, vamos precisar "quebrar" o nosso texto em palavras.

Para implementar essa funcionalidade, vamos utilizar o SPACY para separar o nosso texto em tokens.

In [ ]:
#!pip install --upgrade spacy
!python -m spacy download pt_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 31.6 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

Com a biblioteca SPACY carregada e dado um tamanho de janela de análise, vamos construir as linhas do nosso dataset.

O tamanho da janela de análise representa o número de palavras/tokens que a nossa rede neural vai receber como entrada para tentar prever a próxima palavra. Logo, para definir cada linha do nosso dataset, nós vamos definir cada linha/amostra com uma entrada de N (tamanho da janela) palavras/tokens e a classe dessa linha vai ser a próxima palavra.

Vamos implementar uma estratégia de algorítmo de janela deslizante para construir esse dataset.

In [ ]:
import numpy as np

import spacy

pln = spacy.load("pt_core_news_sm", disable=[
    "morphologizer", "senter", "attribute_ruler", "ner"])

window_size = 30

X = []
labels = []

for text in corpus:
  doc = list(pln(text))
  tokens = [ token.text for token in doc ]

  for i in range(0, len(tokens)-1):
    context = tokens[max(i-window_size, 0):i]
    label = tokens[i]

    X.append(' '.join(context))
    labels.append(label)


X = np.array(X, dtype="object")
print(X.shape)
print(X[:10])

print(labels[:10])

(49520,)
['' 'anexo' 'anexo da' 'anexo da resolução' 'anexo da resolução cogep'
 'anexo da resolução cogep /' 'anexo da resolução cogep / utfpr'
 'anexo da resolução cogep / utfpr nº'
 'anexo da resolução cogep / utfpr nº 110'
 'anexo da resolução cogep / utfpr nº 110 de']
['anexo', 'da', 'resolução', 'cogep', '/', 'utfpr', 'nº', '110', 'de', '19']


A camada de Softmax da nossa rede no Keras não aceita palavras como classes de saída. Por isso vamos precisar codificar a nossa saída, utilizando o padrão One-Hot-Encoding, como um vetor com N valores. Sendo que cada um desses N valores representa uma palavra que o nosso modelo pode gerar como saída.

A biblioteca Keras disponibiliza a função to_categorical que facilita a implementação dessa conversão.

In [ ]:
from keras.utils import to_categorical


word_index = list(set(labels))
labels_index = [word_index.index(label) for label in labels]
y = to_categorical(labels_index)

print(y.shape)
print(y[:10])

(49520, 2701)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


##Configuração do Modelo
Após definirmos o nosso dataset, nós podemos configurar o nosso modelo de Rede Neural. Especificamente para esse exemplo, vamos definir uma rede com as seguintes camadas:
* Vetorização de textos, para gerar as sequências de entrada na rede, conforme vimos na aula passada;
* Embedding, contendo as representações vetoriais das palavras do nosso córpus;
* Duas camadas de LSTM;
* Uma camada Dense;
* Uma camada Dense com ativação de tipo Softmax.

Nesse modelo, nós utilizamos uma configuração de rede profunda. Observem que dada a maior complexidade do problema a ser tratado, estamos utilizando uma rede também mais complexa. A camada de Embeddings deve utilizar 300 dimensões para representar os tokens; temos duas camadas LSTM com 300 neurônios cada; uma camada Dense também com 300 neurônios; e uma camada Softmax, com 1882 neurônios, um para cada palavra que pode ser gerada pelo nosso modelo.

Nessa rede, nós também utilizamos parâmetros de Dropout para reduzir chances de overfitting nas camadas LSTM.

Ao compilar o modelo, nós utilizamos uma função de perda diferente, relacionada ao fato de que o nosso não é mais binário, mas considera a possibilidade de múltiplas categorias.

In [ ]:
from keras.layers import Embedding, LSTM, Dense, TextVectorization
from keras.models import Sequential
from keras.optimizers import AdamW


VOCAB_SIZE = 20000
MAX_SEQUENCE_SIZE = window_size
NEURONS = 300
EPOCHS = 5
EMBEDDING_DIM = 300

vectorization_layer = TextVectorization(
    VOCAB_SIZE, output_sequence_length=MAX_SEQUENCE_SIZE)
vectorization_layer.adapt(X)

model = Sequential()
model.add(vectorization_layer)
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM))
model.add(LSTM(NEURONS, return_sequences=True))
model.add(LSTM(NEURONS))
model.add(Dense(NEURONS, activation='relu'))
model.add(Dense(len(word_index), activation='softmax'))

model.compile(optimizer=AdamW(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(X, y, epochs=EPOCHS, validation_split=0.1)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization_3                 │ ?                           │     0 (unbuilt) │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 28s 16ms/step - accuracy: 0.0671 - loss: 6.2629 - val_accuracy: 0.0868 - val_loss: 6.2898
Epoch 2/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.1032 - loss: 5.3736 - val_accuracy: 0.1008 - val_loss: 6.1247
Epoch 3/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 40s 16ms/step - accuracy: 0.1376 - loss: 4.6897 - val_accuracy: 0.1315 - val_loss: 6.2681
Epoch 4/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.2079 - loss: 4.0970 - val_accuracy: 0.1430 - val_loss: 6.5153
Epoch 5/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.3103 - loss: 3.4223 - val_accuracy: 0.1708 - val_loss: 6.8629


Sobre os resultados de treinamento do nosso modelo, é possível observarmos que a acurácia de treinamento e de validação ficaram baixas. Isso pode ser resultado da complexidade do problema de geração de palavras. O nosso modelo pode prever até 1882 palavras, e garantir uma porcentagem elevada de acertos torna-se mais difícil.

Em seguida, implementamos uma função de geração de texto. Essa função recebe como entrada o nosso modelo treinado, um conjunto de palavras como entrada, o número de palavras que desejamos gerar, um tamanho máximo da sequência que o nosso modelo suporta e o índice de palavras.

O conjunto de palavras dado como entrada deve ser passado para o nosso modelo e usado para gerar palavras que podem ser incluídas ao final da sentença. O nosso modelo gera como saída um conjunto de probabilidades de ocorrências associadas à índices numéricos (Softmax), sendo que o índice numérico com maior probabilidade representa a palavra que o nosso modelo prediz como mais provável de ser incluída no nosso texto. Esses índices são referentes ao índice de palavras (word_index) que pode ser utilizado para transformar o índice numérico em palavra.

Toda vez que uma palavra é gerada, nós incluímos essa palavra como entrada para gerar a próxima palavra do nosso texto.

Como ainda estamos trabalhando com modelos simples, treinados com poucos parâmetros e poucas épocas, nós configuramos na nossa função a geração de 5 textos distintos entre os mais prováveis que poderiam ser gerados.

In [ ]:
def generate_text (model, input, num_words, max_sequence_size, word_index):
  outcomes = []

  generated_words = []
  context = input.split()

  diff = max_sequence_size - len(context)
  initial_context = ['' for i in range(diff)] + context[-max_sequence_size:]  # left padding
  x_test = ' '.join(initial_context).lstrip()

  pred = model.predict(np.array([x_test], dtype="object"), verbose=0)
  most_probable = [ word_index[i] for i in np.argsort(pred[0])[-5:] ]

  print(input)

  for next in most_probable:
    generated_words = [next]
    context = initial_context[1:]
    context.append(next)

    for i in range(num_words):
      x_test = ' '.join(context).lstrip()

      pred = model.predict(np.array([x_test], dtype="object"), verbose=0)
      next_word = word_index[np.argmax(pred[0])]
      generated_words.append(next_word)
      context = context[1:]
      context.append(next_word)

    print(' - ' + ' '.join(generated_words))


input = 'Convalidação é um procedimento para que os alunos possam'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'Como atividades completares podem ser realizadas'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'O coeficiente é utilizado para determinar se o aluno'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'Trabalho de Conclusão de Curso pode ser realizado no período'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'Disciplinas optativas podem ser realizadas no momento em que'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

print(' ----------- ')

generate_text(model, X[90], 20, MAX_SEQUENCE_SIZE, word_index)
generate_text(model, X[242], 20, MAX_SEQUENCE_SIZE, word_index)



Convalidação é um procedimento para que os alunos possam
 - aaes de concluído conjunto de editais de extensão de extensão de cursos de graduação e a partir de ser de ser
 - interveniência do curso de editais de legislação concluído atividades máximo máximo . . . . . . . . . .
 - cursos de graduação e o prazo máximo máximo estabelecido em atividades . . . . . . . . . .
 - programas de soma de legislação tenha concluído período letivo . . . . . . . . . . . .
 - concluído período letivo em atividades de legislação vigente o estudante de cursos de graduação e a partir de partir e a
Como atividades completares podem ser realizadas
 - oficialmente em um professor máximo mínimo de editais de legislação vigente a partir de indicadores profissional de data de partir e
 - matriculado em cursos de graduação e o estudante de crença de extensão de extensão de cursos de graduação e de cursos
 - o estágio a ser tenha de concluído conjunto de editais de ser profissional a partir de 1996 a 1996 e de
 -

##Embeddings Pré-Treinados e Transfer Learning
Com o objetivo de melhorar o processo de geração de texto, nós podemos incluir Embeddings pré-treinados para inicializarmos a camada de Embeddings do nosso modelo. Nesse exemplo, vamos carregar Embeddings pré-treinados com o modelo Skip-Gram de 300 dimensões.

Inicialmente, vamos carregar os embeddings pré-treinados em nosso ambiente.

In [ ]:
import zipfile


# download the embeddings
def download_zip (url, emb_filename):
  if (os.path.isfile(f'./{emb_filename}')):
    print('Arquivo já existente no Runtime... Tudo OK')
  else:
    response = requests.get(url)

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
            zip_ref.extractall('./')
            print("Download concluído e arquivo pronto...")
    else:
        print("Failed to download the zip file.")


zip_url = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s300.zip"
zip_filename = zip_url.split('/')[-1]
emb_filename = zip_filename.replace('.zip', '.txt')
download_zip (zip_url, emb_filename)

Download concluído e arquivo pronto...


Em seguida, vamos utilizar o vetorizador de textos para gerar o vocabulário de palavras que o nosso corpus utiliza.

In [ ]:
MAX_SIZE_VOCAB = 10000

vectorization_layer = TextVectorization(
    MAX_SIZE_VOCAB, output_sequence_length=MAX_SEQUENCE_SIZE)
vectorization_layer.adapt(corpus)
vocab = vectorization_layer.get_vocabulary()

print(len(vocab))
print(vocab[:20])

2727
['', '[UNK]', 'de', 'a', 'art', 'e', 'o', 'do', 'da', 'em', 'no', 'que', 'curso', 'as', 'para', 'ou', 'estudante', 'utfpr', 'atividades', 'curriculares']


Tendo o vocabulário definido, vamos, para cada palavra desse vocabulário, carregar seu vetor utilizando a biblioteca Gensim. Com os vetores das palavras, vamos construir a matriz de pesos que deve ser utilizada para inicializar a nossa camada de Embeddings dentro da nossa rede neural.

Esses passos são similares aos passos que utilizamos em aulas passadas para carregar os Embeddings.

In [ ]:
from gensim.models import KeyedVectors


vectors = KeyedVectors.load_word2vec_format(emb_filename)


def get_weight_matrix (vocab, vectors):
  weights_matrix = []
  _, embedding_dim = vectors.vectors.shape

  for word in vocab:
    if word in vectors:
      weights_matrix.append(vectors[word])
    else:
      weights_matrix.append(np.random.rand(embedding_dim))


  return np.array(weights_matrix, dtype='float32')


weights_matrix = get_weight_matrix(vocab, vectors)
print(weights_matrix.shape)
print(weights_matrix)


(2727, 300)
[[ 0.20778722  0.58353364  0.36505863 ...  0.12947296  0.8915747
   0.9080447 ]
 [ 0.861984    0.28851813  0.5303404  ...  0.9152925   0.32804355
   0.825434  ]
 [-0.103755    0.070214   -0.018556   ...  0.043787    0.072865
  -0.100664  ]
 ...
 [ 0.17013484  0.6039773   0.4264811  ...  0.8406354   0.03417933
   0.13831428]
 [ 0.241192    0.79949486  0.2419448  ...  0.9242134   0.8573463
   0.93878764]
 [ 0.17085135  0.36281216  0.35430688 ...  0.28933045  0.40762407
   0.73001516]]


Em seguida, vamos inicializar a matriz de pesos na camada de Embeddings e treinar nosso modelo.

In [ ]:
model = Sequential()
model.add(vectorization_layer)
model.add(Embedding(len(vocab), EMBEDDING_DIM, weights=[weights_matrix]))
model.add(LSTM(NEURONS, return_sequences=True))
model.add(LSTM(NEURONS))
model.add(Dense(NEURONS, activation='relu'))
model.add(Dense(len(word_index), activation='softmax'))

model.compile(optimizer=AdamW(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, epochs=EPOCHS, validation_split=0.1)

model.summary()

Epoch 1/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.0795 - loss: 6.1291 - val_accuracy: 0.1068 - val_loss: 6.0816
Epoch 2/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.1164 - loss: 4.9922 - val_accuracy: 0.1339 - val_loss: 6.0057
Epoch 3/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.2103 - loss: 4.1370 - val_accuracy: 0.1492 - val_loss: 6.0948
Epoch 4/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.3658 - loss: 3.1676 - val_accuracy: 0.1753 - val_loss: 6.7596
Epoch 5/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.4845 - loss: 2.4720 - val_accuracy: 0.1721 - val_loss: 7.1718


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization_4                 │ (None, 30)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_4 (Embedding)              │ (None, 30, 300)             │         818,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 30, 300)             │         721,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 300)                 │         721,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 300)                 │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2701)                │         813,001 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,491,405 (36.21 MB)

 Trainable params: 3,163,801 (12.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,327,604 (24.14 MB)

Em seguida, podemos utilizar o modelo treinado para gerar os mesmo textos que foram testados anteriormente.

In [ ]:
input = 'Convalidação é um procedimento para que os alunos possam'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'Como atividades completares podem ser realizadas'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'O coeficiente é utilizado para determinar se o aluno'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'Trabalho de Conclusão de Curso pode ser realizado no período'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

input = 'Disciplinas optativas podem ser realizadas no momento em que'
generate_text(model, input, 20, MAX_SEQUENCE_SIZE, word_index)

print(' ----------- ')

generate_text(model, X[90], 20, MAX_SEQUENCE_SIZE, word_index)
generate_text(model, X[242], 20, MAX_SEQUENCE_SIZE, word_index)


Convalidação é um procedimento para que os alunos possam
 - iv realizado de comissão de caráter escritas ou reitoria de graduação e educação profissional prograd a lei nº de de lei
 - recorrer à atividades de extensão obrigatório de caráter escritas ou uce a lei nº 9.394 de 20 de dezembro de 1996
 - analisados pela comissão da realização de ensino e não e seja 01 de intercâmbio de extensão e a lei nº de
 - disponibilizar atividades complementares ou deverá ser analisados e direta as comunidades externas à universidade tecnológica federal do paraná utfpr . .
 - ser feita pelo estudante poderá ser analisados e atuando em modalidades do período de intercâmbio internacional e obrigatório e a lei
Como atividades completares podem ser realizadas
 - pelo colegiado de curso de graduação da utfpr devem ser apresentado no ppc deverá estabelecer conforme modelo definidos pelo sistema nacional
 - por aaes a realização de extensão responsável pelas atividades presencial de pesquisa presencial e tur